In [ ]:
from datetime import date, timedelta
import pytz

import pandas as pd

from app.data.utils.bigquery import BigQuery

bq = BigQuery()

In [ ]:
yesterday = date.today() - timedelta(days=1)
last_datetime = datetime(yesterday.year, yesterday.month, yesterday.day, 23, 59, 59, 999999)

In [ ]:
def format_timestamp(timestamp):
    string = last_datetime.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + '-0400'
    return string

last_datetime_string = format_timestamp(last_datetime)

In [ ]:
last_timestamp_dataframe = pd.DataFrame({
    'site': 'MLA',
    'ds': last_datetime.date(),
    'LAST_TIMESTAMP_EXECUTION': [last_datetime_string]
})

In [ ]:
sql = '''
DROP TABLE meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_TIMESTAMP_EXECUTIONS;
CREATE TABLE meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_TIMESTAMP_EXECUTIONS
(site STRING, ds DATE, LAST_TIMESTAMP_EXECUTION STRING)
PARTITION BY ds
CLUSTER BY site
;
'''

bq.run_query(sql)

In [ ]:
sql = '''
DELETE FROM meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_TIMESTAMP_EXECUTIONS WHERE 1=1
;
'''

bq.run_query(sql)

In [ ]:
bq.database.client.load_table_from_dataframe(
    last_timestamp_dataframe,
    'meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_TIMESTAMP_EXECUTIONS'
)